# Advent of Code 2018
We have to save christmas by solving 50 puzzles and going back in time.

First I define some utility functions, e.g. to retrieve the input file for a given day using the sessioncookie stored in the file `cookie`.

In [1]:
import requests
import os
def Input(day):
    if not os.path.exists(f'day{day}.input'):
        print(f'downloading input file for day {day}')
        with open('cookie') as f:
            cookie = f.read().strip()
        resp = requests.get(f'https://adventofcode.com/2018/day/{day}/input',
                            cookies={'session': cookie})
        if resp.status_code == 200:
            with open(f'day{day}.input', 'w') as f:
                f.write(resp.text)
        else:
            print(f'Coockie might be expired: {res.status_code} {res.reason}')
    with open(f'day{day}.input') as f:
        return f.read().split()

## Day 1
We have a frequency-changing device, so first we look at the end-frequency after all changes in the file are applied to the start-frequency of 0. The second question is, which frequency is reached twice first, possibly having to loop the input multiple times

In [2]:
def end_frequency(input):
    return sum([ int(l) for l in input])
assert end_frequency(['+1']) == 1
assert end_frequency([-1, 2,-2,5]) == 4
end_frequency(Input(1))

490

In [3]:
def find_duplicate_frequency(changes):
    reached_frequencies = set([0])
    current_frequency = 0
    for i in range(10**100):
        for l in changes:
            current_frequency += int(l)
            if current_frequency in reached_frequencies:
                #print(f'iteration: {i}, {current_frequency}')
                return current_frequency
            reached_frequencies.add(current_frequency)
assert find_duplicate_frequency([1,-1]) == 0
assert find_duplicate_frequency([4, -2]) == 4
find_duplicate_frequency(Input(1))

70357